## CBOE SKEW analysis

In [82]:
import sys,os

if  not os.path.abspath('./') in sys.path:
    sys.path.append(os.path.abspath('./'))
if  not os.path.abspath('../') in sys.path:
    sys.path.append(os.path.abspath('../'))
sys.path.append(os.path.abspath('../../dashgrid'))
sys.path.append(os.path.abspath('../../dashgrid/dashgrid'))
# print(sys.path)
from dashgrid import dgrid
import dash_core_components as dcc
import traceback

from volgrid import create_voltables
import plotly.graph_objs as go
from plotly.offline import iplot
from plotly.offline import  init_notebook_mode, iplot
init_notebook_mode(connected=True)
import numpy as np
import pandas as pd
from pandas.tseries.offsets import BDay
import pandas_datareader.data as pdr
import datetime
import pytz

#  do rest of imports
import dash
import dash_html_components as html
from dash.dependencies import Input, Output,State
import yfinance as yf
import pathlib
import pg_pandas as pg

### Paremeters to calculate SKEW

In [2]:
def get_rate(num_months_for_rate=1,start_datetime=None):
    # see if you can get Libor from the FRED API
    if start_datetime is None:
        n = datetime.datetime.now() - datetime.timedelta(7)
    else:
        n = start_datetime #- datetime.timedelta(14)
    y = n.year
    m = n.month
    d = n.day
    beg = '%04d-%02d-%02d' %(y,m,d)
    ed = n  + datetime.timedelta(1)
    y = ed.year
    m = ed.month
    d = ed.day
    eds = '%04d-%02d-%02d' %(y,m,d)
    df = pdr.DataReader(f'USD{num_months_for_rate}MTD156N', "fred", f'{beg}', f'{eds}')
    fixed_rate = float(df.iloc[len(df)-1][f'USD{num_months_for_rate}MTD156N'])/100
    return fixed_rate


In [3]:
SKEW_TICKER = '^SPX' # yahoo symbol
THRESHOLD_BID = .1 # minimum bid value when screening options
DELTA_K = 5 # strike difference between most of the options that get used to calculate SKEW
TIMEZONE = 'US/Eastern'
TRADE_DATE = 20191224
EXP_NEAR_YYYYMMDD = 20200116
EXP_FAR_YYYYMMDD = 20200220
INTEREST_RATE = (get_rate(1) + get_rate(2))/2 # calculate at run time ( this will be the average of the 1 month and 2 month libor rates from FRED)

OUTPUT_COLS = ['contractSymbol','strike','mid','dte_pct','ert',
                'forward_price','deltak','p1','p2','p3','e1','e2','e3']


In [4]:
df_skew = pd.read_csv('https://www.cboe.com/publish/scheduledtask/mktdata/datahouse/skewdailyprices.csv',header=1)

In [5]:
df_skew['date_split'] = df_skew.Date.apply(lambda s:s.split('/'))
df_skew['yyyymmdd'] = df_skew.date_split.apply(lambda s:int(s[2])*100*100 + int(s[0])*100 + int(s[1]))
df_skew

,Date,SKEW,Unnamed: 2,Unnamed: 3,date_split,yyyymmdd
0,1/2/1990,126.09,NaN,NaN,"[1, 2, 1990]",19900102
1,1/3/1990,123.34,NaN,NaN,"[1, 3, 1990]",19900103
2,1/4/1990,122.62,NaN,NaN,"[1, 4, 1990]",19900104
3,1/5/1990,121.27,NaN,NaN,"[1, 5, 1990]",19900105
4,1/8/1990,124.12,NaN,NaN,"[1, 8, 1990]",19900108
...,...,...,...,...,...,...
7539,12/18/2019,144.56,NaN,NaN,"[12, 18, 2019]",20191218
7540,12/19/2019,150.14,NaN,NaN,"[12, 19, 2019]",20191219
7541,12/20/2019,147.17,NaN,NaN,"[12, 20, 2019]",20191220
7542,12/23/2019,143.91,NaN,NaN,"[12, 23, 2019]",20191223


In [6]:
spx = yf.Ticker(SKEW_TICKER)
spx.options

('2019-12-26',
 '2019-12-29',
 '2019-12-30',
 '2020-01-02',
 '2020-01-05',
 '2020-01-07',
 '2020-01-09',
 '2020-01-12',
 '2020-01-14',
 '2020-01-16',
 '2020-01-20',
 '2020-01-21',
 '2020-01-23',
 '2020-01-26',
 '2020-01-30',
 '2020-02-06',
 '2020-02-13',
 '2020-02-20',
 '2020-02-27',
 '2020-03-05',
 '2020-03-19',
 '2020-03-30',
 '2020-04-16',
 '2020-04-29',
 '2020-05-14',
 '2020-05-28',
 '2020-06-18',
 '2020-06-29',
 '2020-09-17',
 '2020-09-18',
 '2020-09-29',
 '2020-10-15',
 '2020-11-19',
 '2020-12-17',
 '2021-01-14',
 '2021-03-18',
 '2021-06-17',
 '2021-12-16',
 '2026-03-19')

In [7]:
def dt_from_yyyymmdd(yyyymmdd,hour=0,minute=0,timezone=TIMEZONE):
    y = int(str(yyyymmdd)[0:4])
    m = int(str(yyyymmdd)[4:6])
    d = int(str(yyyymmdd)[6:8])  
    return datetime.datetime(y,m,d,hour,minute,tzinfo=pytz.timezone(timezone))
def get_dte_pct(trade_yyyymmdd,expiry_yyyymmdd):
    dt_td = dt_from_yyyymmdd(trade_yyyymmdd)
    dt_xp = dt_from_yyyymmdd(expiry_yyyymmdd)
    return ((dt_xp - dt_td).days + 1)/365



In [8]:
ed = str(EXP_FAR_YYYYMMDD)
dd = f'{ed[0:4]}-{ed[4:6]}-{ed[6:8]}'
c = yf.Ticker('^SPX')
df_t = c.option_chain(dd)
              

In [9]:
df_tp = df_t.puts[(df_t.puts.contractSymbol.str.slice(0,4)=='SPX2')]
df_tp[df_tp.strike>=3220.0].iloc[:3]

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency
228,SPX200221P03220000,2019-12-26 18:40:44,3220.0,49.36,49.4,49.8,-5.66,-10.287168,4.0,164,0.109693,False,REGULAR,USD
230,SPX200221P03230000,2019-12-26 18:40:44,3230.0,52.69,52.7,53.1,-5.91,-10.085324,6.0,9,0.106679,False,REGULAR,USD
232,SPX200221P03240000,2019-12-26 18:40:44,3240.0,56.39,56.3,56.7,-5.91,-9.486357,3.0,45,0.103708,True,REGULAR,USD


In [10]:
df_tc = df_t.calls[(df_t.calls.bid>.1) & (df_t.calls.contractSymbol.str.slice(0,4)=='SPX2')]
df_tc[~df_tc.inTheMoney].iloc[:3]

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency
102,SPX200221C03235000,2019-12-26 19:27:28,3235.0,56.0,55.7,56.2,5.849998,11.665002,43.0,224,0.110746,False,REGULAR,USD
105,SPX200221C03250000,2019-12-26 17:12:41,3250.0,45.9,46.7,47.1,3.630001,8.587653,30.0,20884,0.106626,False,REGULAR,USD
107,SPX200221C03260000,2019-12-26 17:01:35,3260.0,41.1,41.2,41.6,3.599998,9.599996,7.0,525,0.104253,False,REGULAR,USD


In [11]:
import pdb
def get_valid_series_from_yfinance(trade_date_yyyymmdd,expiry_yyyymmdd,ticker=SKEW_TICKER,threshold_bid=THRESHOLD_BID,interest_rate=INTEREST_RATE,deltak=DELTA_K):
#     # ****** Step 01: create interest rate to use if necessary
    ir = interest_rate
#     if ir is None:
#         ir  = (get_rate(1) + get_rate(2))/2
    
    # ******  Step 02:  get yfinance contract, and options series
    contract = yf.Ticker(ticker)
    # reformat expiry_yyyymmdd into something like 2019-12-20
    ed = str(expiry_yyyymmdd)
    date_string = f'{ed[0:4]}-{ed[4:6]}-{ed[6:8]}'
    # get the options chain from yahoo finance
    df_opt = contract.option_chain(date_string)
    
    # ****** Step 03: figure out forward price
    df_spx_c = df_opt.calls[(df_opt.calls.bid>threshold_bid) & (df_opt.calls.contractSymbol.str.slice(0,4)=='SPX2')]
    df_spx_c['cp'] = 'c'
    df_spx_p = df_opt.puts[(df_opt.puts.bid>threshold_bid) & (df_opt.puts.contractSymbol.str.slice(0,4)=='SPX2')]
    df_spx_p['cp'] = 'p'
    
    # ******  Step 03: extract out the ATM and the out of the money calls
    df_spx_c = df_spx_c[~df_spx_c.inTheMoney]
    df_spx_c = df_spx_c.sort_values('strike')
    df_spx_c.index = range(len(df_spx_c))
    
    first_call_strike = df_spx_c.iloc[0].strike
    # save the lowest_itm_put for later calc of forward price
    call_strikes = df_spx_c.strike.values
    lowest_itm_put = df_spx_p[(df_spx_p.strike>=first_call_strike) & df_spx_p.strike.isin(call_strikes)].iloc[0]
    
    
    # ******  Step 04:  extract out of the money puts
    #    get atm option strike (which is first call)
    df_spx_p = df_spx_p[~df_spx_p.inTheMoney]
    df_spx_p = df_spx_p.sort_values('strike')
    df_spx_p.index = range(len(df_spx_p))

    
    # ******  Step 05: merge puts and alls into one dataframe
    df_ret = df_spx_p.copy()
    df_ret = df_ret.append(df_spx_c)
    df_ret = df_ret.sort_values(['cp','strike'])
    df_ret.index = range(len(df_ret))
    df_ret['expiry_yyyymmdd'] = expiry_yyyymmdd
    df_ret['trade_date_yyyymmdd'] = trade_date_yyyymmdd

    # ******  Step 06: Add mid price, expiry, dte_pct, ert, forward price     
    df_ret['mid'] = (df_ret.bid + df_ret.ask)/2
    dte_pct = get_dte_pct(trade_date_yyyymmdd,expiry_yyyymmdd)
    df_ret['dte_pct'] = dte_pct
    ert = np.exp(dte_pct * ir)
    df_ret['ert'] = ert
    # find forward price by using the lowest strike call in df_ret 
    #     (what the SKEW whitepaper calls the At The Money option)
    # get ATM call midpoint
    atm_strike = lowest_itm_put.strike 
    print(atm_strike)
    atm_call = df_ret[(df_ret.strike==atm_strike) & (df_ret.cp=='c')].iloc[0]
    atm_call_price = (atm_call.bid + atm_call.ask) / 2 
    atm_put = lowest_itm_put
    atm_put_price = (atm_put.bid + atm_put.ask) / 2 
    forward_price = ert*(atm_call_price - atm_put_price) + atm_strike
    df_ret['forward_price'] = forward_price
    df_ret['k_over_fp'] = df_ret.strike / forward_price
    df_ret['deltak'] = deltak
    # create p1 unit values
    df_ret['p1']= df_ret.apply(lambda r: r.deltak / r.strike**2 * r.mid,axis=1)
    df_ret['p2'] = df_ret.apply(lambda r: 2 * r.p1 *(1-np.log(r.k_over_fp)),axis=1)
    df_ret['p3'] = df_ret.apply(lambda r: 3 * r.p1 * (2*np.log(r.k_over_fp) - np.log(r.k_over_fp)**2),axis=1)
    e1 = -(1+np.log(forward_price/atm_strike) - forward_price/atm_strike)
    e2 = 2 * np.log(atm_strike/forward_price) * (forward_price/atm_strike - 1) + 1/2 * np.log(atm_strike/forward_price)**2
    e3 = 3 * np.log(atm_strike/forward_price)**2 * (1/3 * np.log(atm_strike/forward_price) - 1 + forward_price/atm_strike)
    df_ret['e1'] = e1
    df_ret['e2'] = e2
    df_ret['e3'] = e3
    return df_ret
    

In [12]:
df_series_near = get_valid_series_from_yfinance(TRADE_DATE,EXP_NEAR_YYYYMMDD)
df_series_far = get_valid_series_from_yfinance(TRADE_DATE,EXP_FAR_YYYYMMDD)


/Users/bperlman1/Virtualenvs3/dashrisk3/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/bperlman1/Virtualenvs3/dashrisk3/lib/python3.6/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



3235.0
3250.0


In [13]:
df_series_far[df_series_far.cp=='c'].iloc[0:1]

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,...,ert,forward_price,k_over_fp,deltak,p1,p2,p3,e1,e2,e3
0,SPX200221C03235000,2019-12-26 19:27:28,3235.0,56.0,55.7,56.2,5.849998,11.665002,43.0,224,...,1.002938,3236.309896,0.999595,5,0.000027,0.000053,-6.494342e-08,0.000009,-0.000027,-1.499595e-07


In [14]:
df_series_near[df_series_near.cp=='p'].iloc[-1:]

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,...,ert,forward_price,k_over_fp,deltak,p1,p2,p3,e1,e2,e3
116,SPX200117P03230000,2019-12-26 18:39:04,3230.0,26.46,26.7,26.9,-6.220001,-19.03305,190.0,223.0,...,1.001194,3236.301552,0.998053,5,0.000013,0.000026,-1.503473e-07,8.091485e-08,-2.427771e-07,1.302150e-10


### sum up p1's p2's and p3's and computer e1, e2, and e3', and create SKEW

In [15]:
def create_skew(df_series):
    e1 = df_series.iloc[0].e1
    e2 = df_series.iloc[0].e2
    e3 = df_series.iloc[0].e3
    ert = df_series.iloc[0].ert
    p1 = ert * -1 * df_series.p1.sum() + e1
    p2 = ert * df_series.p2.sum() + e2
    p3 = ert * df_series.p3.sum() + e3
    S =  (p3 - 3*p1*p2 + 2*p1**3) / (p2 - p1**2)**(3/2)
    SKEW = 100 - 10*S
    return SKEW

In [16]:
Tnear = df_series_near.iloc[0].dte_pct
Tfar = df_series_far.iloc[0].dte_pct
T30 = 30/365
w = (Tfar-T30)/(Tfar-Tnear)
skew_near = create_skew(df_series_near)
skew_far = create_skew(df_series_far)
final_skew = w*skew_near + (1-w) * skew_far
print(final_skew,skew_near,skew_far,w)

150.48336252300197 150.5611851497352 150.1072198271248 0.8285714285714285


### now do multiple days from Oct 2015

In [17]:
def get_valid_series_from_csv_file_df(df,threshold_bid=THRESHOLD_BID,interest_rate=INTEREST_RATE,deltak=DELTA_K):
    '''
    df: a DataFrame containing options settlements for a single day, and a single expiry
    '''
    # ****** Step 01: create interest rate to use if necessary
    ir = interest_rate
#     if ir is None:
#         ir  = (get_rate(1) + get_rate(2))/2
    
    df_ret = df.copy()
    trade_date_yyyymmdd = df_ret.trade_date_yyyymmdd.iloc[0]
    expiry_yyyymmdd = df_ret.expiry_yyyymmdd.iloc[0]
    assert(len(df_ret.expiry_yyyymmdd.unique())==1)
    # ******  Step 02: Add mid price, expiry, dte_pct, ert, forward price     
    df_ret['mid'] = (df_ret.bid + df_ret.ask)/2
    dte_pct = get_dte_pct(trade_date_yyyymmdd,expiry_yyyymmdd)
    df_ret['dte_pct'] = dte_pct
    ert = np.exp(dte_pct * ir)
    df_ret['ert'] = ert
    lowest_atm_call = df_ret[(df_ret.underlying_last<df_ret.strike) & (df_ret.cp=='c')].sort_values('strike').iloc[0]
    atm_call_price = lowest_atm_call.mid
    atm_strike = lowest_atm_call.strike
    atm_put_price = df_ret[(df_ret.strike==atm_strike) & (df_ret.cp=='p')].iloc[0].mid
    forward_price = ert*(atm_call_price - atm_put_price) + atm_strike
    
    # ******  Step 03: Limit options to out of the money puts, the ATM call and out of the money calls
    df_ret_c = df_ret[(df_ret.cp=='c') & (df_ret.strike>=atm_strike)]
    df_ret_p = df_ret[(df_ret.cp=='p') & (df_ret.strike<atm_strike)]
    df_ret = df_ret_c.append(df_ret_p).sort_values(['cp','strike'])
    # ******  Step 04: make sure options have minimum bid
    df_ret = df_ret[df_ret.bid>threshold_bid]
    
    
    df_ret['forward_price'] = forward_price
    df_ret['k_over_fp'] = df_ret.strike / forward_price
    df_ret['deltak'] = deltak
    # create p1 unit values
    df_ret['p1']= df_ret.apply(lambda r: r.deltak / r.strike**2 * r.mid,axis=1)
    df_ret['p2'] = df_ret.apply(lambda r: 2 * r.p1 *(1-np.log(r.k_over_fp)),axis=1)
    df_ret['p3'] = df_ret.apply(lambda r: 3 * r.p1 * (2*np.log(r.k_over_fp) - np.log(r.k_over_fp)**2),axis=1)
    e1 = -(1+np.log(forward_price/atm_strike) - forward_price/atm_strike)
    e2 = 2 * np.log(atm_strike/forward_price) * (forward_price/atm_strike - 1) + 1/2 * np.log(atm_strike/forward_price)**2
    e3 = 3 * np.log(atm_strike/forward_price)**2 * (1/3 * np.log(atm_strike/forward_price) - 1 + forward_price/atm_strike)
    df_ret['e1'] = e1
    df_ret['e2'] = e2
    df_ret['e3'] = e3

    return df_ret
    

In [18]:
def add_days_to_yyyymmdd(yyyymmdd,days_to_add):
    s = str(yyyymmdd)
    y = int(s[0:4])
    m = int(s[4:6])
    d = int(s[6:8])
    dt = datetime.datetime(y,m,d) + datetime.timedelta(days_to_add)
    ret = int(dt.year)*100*100 + int(dt.month)*100 + int(dt.day)
    return ret

In [19]:
home =  pathlib.Path.home()
df_spx_oct_2015= pd.read_csv(f'{home}/downloads/spx_20151001_to_20151030.csv')
df_spx_oct_2015 = df_spx_oct_2015[df_spx_oct_2015.underlying=='SPX']
df_spx_oct_2015['cp'] = df_spx_oct_2015['type'].apply(lambda v:v[0])
df_spx_oct_2015['trade_date_yyyymmdd'] = df_spx_oct_2015.quotedate.apply(lambda s:s.split('/'))
df_spx_oct_2015['trade_date_yyyymmdd'] = df_spx_oct_2015.trade_date_yyyymmdd.apply(lambda s:(2000 + int(s[2]))*100*100 + int(s[0])*100 + int(s[1]))
df_spx_oct_2015['expiry_yyyymmdd'] = df_spx_oct_2015.expiration.apply(lambda s:s.split('/'))
df_spx_oct_2015['expiry_yyyymmdd'] = df_spx_oct_2015.expiry_yyyymmdd.apply(lambda s:(2000 + int(s[2]))*100*100 + int(s[0])*100 + int(s[1]))

df_spx_oct_2015.tail()


,underlying,underlying_last,exchange,optionroot,optionext,type,expiration,quotedate,strike,last,...,openinterest,impliedvol,delta,gamma,theta,vega,optionalias,cp,trade_date_yyyymmdd,expiry_yyyymmdd
233689,SPX,2084.58,*,SPX171215P02750000,NaN,put,12/15/17,10/30/15,2750,0.00,...,0,0.1993,-0.7836,0.0005,-34.2409,891.7503,SPX171215P02750000,p,20151030,20171215
233690,SPX,2084.58,*,SPX171215P02800000,NaN,put,12/15/17,10/30/15,2800,0.00,...,0,0.2036,-0.7945,0.0005,-33.6388,865.0028,SPX171215P02800000,p,20151030,20171215
233691,SPX,2084.58,*,SPX171215P02900000,NaN,put,12/15/17,10/30/15,2900,0.00,...,0,0.2134,-0.8114,0.0004,-33.0113,821.1380,SPX171215P02900000,p,20151030,20171215
233692,SPX,2084.58,*,SPX171215P03000000,NaN,put,12/15/17,10/30/15,3000,917.55,...,18,0.2238,-0.8241,0.0004,-32.7816,786.1406,SPX171215P03000000,p,20151030,20171215
233693,SPX,2084.58,*,SPX171215P03500000,NaN,put,12/15/17,10/30/15,3500,1585.70,...,69,0.2685,-0.8665,0.0003,-30.9119,654.9595,SPX171215P03500000,p,20151030,20171215


### Get unique quotedate's

In [20]:
trade_dates_yyyymmdd = df_spx_oct_2015.trade_date_yyyymmdd.unique()
dict_df_exp_pair = {}
for d in trade_dates_yyyymmdd:
    first_expiry_cutoff = add_days_to_yyyymmdd(d,10)
    df_spx_1 = df_spx_oct_2015[(df_spx_oct_2015.trade_date_yyyymmdd==d) & (df_spx_oct_2015.expiry_yyyymmdd>first_expiry_cutoff)]
    first_2_expiries = df_spx_1.expiry_yyyymmdd.unique()[:2]
    df_spx_exp1 = df_spx_1[df_spx_1.expiry_yyyymmdd==first_2_expiries[0]]
    df_spx_exp1_with_all_fields = get_valid_series_from_csv_file_df(df_spx_exp1)
    df_spx_exp2 = df_spx_1[df_spx_1.expiry_yyyymmdd==first_2_expiries[1]]
    df_spx_exp2_with_all_fields = get_valid_series_from_csv_file_df(df_spx_exp2)

    Tnear = df_spx_exp1_with_all_fields.iloc[0].dte_pct
    Tfar = df_spx_exp2_with_all_fields.iloc[0].dte_pct
    T30 = 30/365
    w = (Tfar-T30)/(Tfar-Tnear)
    skew_near = create_skew(df_spx_exp1_with_all_fields)
    skew_far = create_skew(df_spx_exp2_with_all_fields)
    final_skew = w*skew_near + (1-w) * skew_far
    actual_skew = df_skew[df_skew.yyyymmdd==d].iloc[0].SKEW
    print(d,actual_skew,final_skew,skew_near,skew_far,w,first_2_expiries[0],first_2_expiries[1])    

    dict_df_exp_pair[d] = {
        'df_spx_1':df_spx_exp1_with_all_fields,
                           'df_spx_2':df_spx_exp2_with_all_fields,
                           'skew_near':skew_near,
                           'skew_far':skew_far,
                           'final_skew':final_skew
    }
    


20151001 124.81 117.39844059933634 114.34108152198098 121.98447921536939 0.6000000000000001 20151016 20151120
20151002 115.75 118.34820738086856 116.04118304643222 121.42423982678366 0.5714285714285714 20151016 20151120
20151005 132.83 120.1387393285725 116.57837360188718 123.50130695933085 0.48571428571428577 20151016 20151120
20151006 129.32 126.83779046545588 124.79011461500916 121.07870213607447 1.5517241379310347 20151120 20151219
20151007 132.39 126.87784773928938 124.86574749647681 120.97568702703917 1.5172413793103448 20151120 20151219
20151008 136.69 130.50116545352395 128.55245412184422 124.51583779193618 1.4827586206896552 20151120 20151219
20151009 134.66 130.42885690963766 129.20898305169317 126.48772598397085 1.4482758620689657 20151120 20151219
20151012 137.25 129.3619368790283 128.46294147545558 125.85585480509464 1.3448275862068966 20151120 20151219
20151013 133.81 129.2603509900469 128.5533084443611 126.2750602415957 1.310344827586207 20151120 20151219
20151014 135.82

### Now use ES contract history from sec_db database

In [27]:
# pg.PgPandas??
pga = pg.PgPandas(dburl='127.0.0.1',username='',password='',databasename='sec_db')

/Users/bperlman1/Virtualenvs3/dashrisk3/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning:

The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.



In [125]:
sql = '''
select * from sec_schema.options_table ot
where ot.settle_date>20180816 
and ot.symbol='CLU18'
;
'''
df_options = pga.get_sql(sql)


sql = '''
select * from sec_schema.underlying_table ft 
where ft.settle_date='20180801' 
and ft.symbol='CLU18'
;
'''
df_futures = pga.get_sql(sql)
df_options['forward_price'] = df_futures.iloc[0].close
df_options['trade_date_yyyymmdd'] = df_options.settle_date

print(df_options)

Empty DataFrame
Columns: [symbol, strike, pc, settle_date, open, high, low, close, adj_close, volume, open_interest, forward_price, trade_date_yyyymmdd]
Index: []


In [88]:
df_options

,symbol,strike,pc,settle_date,open,high,low,close,adj_close,volume,open_interest


### Create date math functions to get options/futures expiry dates

In [120]:
from pandas.tseries.holiday import USFederalHolidayCalendar

bday_us = pd.offsets.CustomBusinessDay(calendar=USFederalHolidayCalendar())

def get_nth_weekday(year,month,target_weekday,nth_occurrence):
    '''
    weekday is the term that assigns numbers from 0 to 6 to the days of the weeks.
    weekday 0 = monday
    '''
    # get dayofweeks of year,month,1
    weekday_01 = datetime.datetime(year,month,1).weekday()
    if weekday_01 <= target_weekday:
        day_of_month_of_first_occurence = target_weekday - weekday_01
        day_of_month_of_nth_occurence = day_of_month_of_first_occurence + 1 + (nth_occurrence - 1) * 7
    else:
        day_of_month_of_nth_occurence = target_weekday - weekday_01 + 1 + (nth_occurrence) * 7 
    return datetime.datetime(year,month,day_of_month_of_nth_occurence)


MONTH_CODES = 'FGHJKMNQUVXZ'
DICT_MONTH_CODE = {MONTH_CODES[i]:i+1 for i in range(len(MONTH_CODES))}
DICT_PRODUCT = {
    'ES':get_ES_expiry,
    'CL':get_CL_expiry,
}


def get_ES_expiry(symbol):
    monthcode_yy = symbol[2:]
    month = DICT_MONTH_CODE[monthcode_yy[0]]
    year = 2000 + int(monthcode_yy[1:])
    return get_nth_weekday(year,month,4,3)

def get_CL_expiry(symbol):
    monthcode_yy = symbol[2:]
    month = DICT_MONTH_CODE[monthcode_yy[0]]
    year = 2000 + int(monthcode_yy[1:])
    month = month -1
    if month<1:
        month = 12
        year = year - 1
    return datetime.datetime(year,month,26) - 7*bday_us# BDay(7)
    
    
def get_expiry(symbol):
    product = symbol[:2]
    f = DICT_PRODUCT[product]
    return f(symbol)

### check dates vs last records in db

In [126]:

for s in ['CL'+m+yy for m in 'FGHJKMNQUVXZ' for yy in ['15','16','17','18']]:
    ge = str(get_expiry(s))
    exp1 = int(ge[0:4])*100*100 + int(ge[5:7])*100 + int(ge[8:10])
    sql = f'''
    select max(ot.settle_date) from sec_schema.options_table ot
    where ot.symbol='{s}' 
    ;
    '''
    df_options = pga.get_sql(sql)
    exp2 = df_options.iloc[0]['max']
    print(s,exp1,exp2,exp1==exp2)
    #assert(exp1==exp2)

CLF15 20141216 20141216 True
CLF16 20151216 20151216 True
CLF17 20161215 20161215 True
CLF18 20171214 20171214 True
CLG15 20150114 20150114 True
CLG16 20160114 20160114 True
CLG17 20170117 20170117 True
CLG18 20180117 20180117 True
CLH15 20150217 20150217 True
CLH16 20160217 20160217 True
CLH17 20170215 20170215 True
CLH18 20180214 20180214 True
CLJ15 20150317 20150317 True
CLJ16 20160317 20160316 False
CLJ17 20170316 20170316 True
CLJ18 20180315 20180315 True
CLK15 20150416 20150416 True
CLK16 20160415 20160415 True
CLK17 20170417 20170417 True
CLK18 20180417 20180417 True
CLM15 20150514 20150514 True
CLM16 20160517 20160517 True
CLM17 20170517 20170517 True
CLM18 20180517 20180517 True
CLN15 20150617 20150617 True
CLN16 20160616 20160616 True
CLN17 20170615 20170615 True
CLN18 20180615 20180615 True
CLQ15 20150716 20150716 True
CLQ16 20160715 20160715 True
CLQ17 20170717 20170717 True
CLQ18 20180717 20180717 True
CLU15 20150817 20150817 True
CLU16 20160817 20160817 True
CLU17 2017081

In [106]:
# import datetime as dt

# from pandas.tseries.holiday import AbstractHolidayCalendar, Holiday, nearest_workday, \
#     USMartinLutherKingJr, USPresidentsDay, GoodFriday, USMemorialDay, \
#     USLaborDay, USThanksgivingDay


# class USTradingCalendar(AbstractHolidayCalendar):
#     rules = [
#         Holiday('NewYearsDay', month=1, day=1, observance=nearest_workday),
#         USMartinLutherKingJr,
#         USPresidentsDay,
#         GoodFriday,
#         USMemorialDay,
#         Holiday('USIndependenceDay', month=7, day=4, observance=nearest_workday),
#         USLaborDay,
#         USThanksgivingDay,
#         Holiday('Christmas', month=12, day=25, observance=nearest_workday)
#     ]


# def get_trading_close_holidays(year):
#     inst = USTradingCalendar()

#     return inst.holidays(dt.datetime(year-1, 12, 31), dt.datetime(year, 12, 31))


# print(get_trading_close_holidays(2014)) 

DatetimeIndex(['2014-01-01', '2014-01-20', '2014-02-17', '2014-04-18',
               '2014-05-26', '2014-07-04', '2014-09-01', '2014-11-27',
               '2014-12-25'],
              dtype='datetime64[ns]', freq=None)


In [70]:
year = 2019
month = 12
target_weekday = 4
nth_occurrence = 1
weekday_01 = datetime.datetime(year,month,1).weekday()
if weekday_01 <= target_weekday:
    day_of_month_of_first_occurence = target_weekday - weekday_01
    day_of_month_of_nth_occurence = day_of_month_of_first_occurence + 1 + (nth_occurrence - 1) * 7
else:
    day_of_month_of_nth_occurence = target_weekday - weekday_01 + 1 + (nth_occurrence) * 7 
datetime.datetime(year,month,day_of_month_of_nth_occurence),weekday_01,day_of_month_of_first_occurence

(datetime.datetime(2019, 12, 6, 0, 0), 6, 4)

In [71]:
get_nth_weekday(2019,12,4,1)

datetime.datetime(2019, 12, 6, 0, 0)

## End

In [40]:
datetime.datetime.now().weekday()

3